# Browse Gaia and Calspec catalogs

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2024/10/05

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
machine_name = os.uname().nodename
path_rubinsimphot = "repos/repos_w_2024_38/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
import getCalspec

In [ ]:
try:
    from gaiaspec import getGaia
except ModuleNotFoundError:
    getGaia = None

## Config 

In [ ]:
FLAG_FILE_ACCESS = True

In [ ]:
cat_fn = "calspec_v2.csv"

In [ ]:
#list_of_targets = ["HD111235","HD144334","HD73495","HD165763"] 

## Read catalog

In [ ]:
if FLAG_FILE_ACCESS: 
    df = pd.read_csv(cat_fn,index_col=0)
    df["GAIA_ED3_Name"] = df["GAIA_ED3_Name"].astype('Int64')
else:
    df = getGaia.get_gaia_calspec_matching()
    df["GAIA_DR3_Name"] = df["GAIA_DR3_Name"].astype('Int64')
df

In [ ]:
#for target in list_of_targets:
#    row = df[df.HD_name == target]
#    print(row)

In [ ]:
if FLAG_FILE_ACCESS: 
    df_s = df[["HD_name","GAIA_ED3_Name"]]
else:
    df_s = df[["HD_name","GAIA_DR3_Name"]] 

In [ ]:
df_s = df_s.dropna(axis=0)

In [ ]:
if FLAG_FILE_ACCESS: 
    list_of_gaia_ids = df_s.GAIA_ED3_Name.unique()
else:    
    list_of_gaia_ids = df_s.GAIA_DR3_Name.unique()

In [ ]:
list_of_gaia_ids = list_of_gaia_ids[~np.isnan(list_of_gaia_ids)]

In [ ]:
list_of_gaia_ids = list_of_gaia_ids.astype(int)

In [ ]:
list_of_gaia_ids

In [ ]:
df_s.reset_index(drop=True, inplace=True)
df_s

## Extract all the spectra

In [ ]:
all_gaia = [] # container for gaia object
all_calspec = []
all_calspec_dict = []
all_spec_dict = [] # container for gaia spectra spectra
all_row = [] # container for info on 
for idx,hd_name in enumerate(df_s.HD_name.values):
    row = df[df["HD_name"] == hd_name]
    flag_calspec = getCalspec.is_calspec(hd_name)
    gaianum = getGaia.get_gaia_name_from_star_name(hd_name)
    print(hd_name, gaianum,"iscalspec = ",flag_calspec)
    g = getGaia.Gaia(hd_name)
    c = getCalspec.Calspec(hd_name)
    try:
        #spec_dict = getGaia.get_gaia_from_query_id(gaianum)
        spec_dict = g.get_spectrum_numpy(hd_name)
        calspec_dict = c.get_spectrum_numpy()
        all_spec_dict.append(spec_dict)
        all_calspec_dict.append(calspec_dict)
        all_gaia.append(g)
        all_calspec.append(c)
        all_row.append(row)
        #pass
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
        #x, y = inst.args     # unpack args
        #print('x =', x)
        #print('y =', y)


        
    
N = len(all_gaia)

## Plot

In [ ]:
for idx,star_gaia in enumerate(all_gaia):  
    star_gaia.plot_spectrum(xscale='linear', yscale='linear') 

In [ ]:
for idx,star_calspec in enumerate(all_calspec):  
    star_gaia.plot_spectrum(xscale='linear', yscale='linear') 

In [ ]:
star_calspec.plot_spectrum?

In [ ]:
for idx,star_calspec in enumerate(all_calspec):  
    #ax =plt.gca()
    #ax.set_xlim(3000.,10000.)
    star_calspec.plot_spectrum(xscale='log', yscale='log') 
    
    


## Check units

In [ ]:
all_spec_dict[0]['WAVELENGTH'].unit 

In [ ]:
all_spec_dict[0]['FLUX'].unit 

In [ ]:
all_calspec_dict[0]['WAVELENGTH'].unit 

In [ ]:
all_calspec_dict[0]['FLUX'].unit 

In [ ]:
all_spec_dict[0]['FLUX'].to(u.erg/(u.cm)**2/u.s/u.angstrom, equivalencies=u.spectral())  

In [ ]:
all_spec_dict[0]['FLUX'].to(u.erg/(u.cm)**2/u.s/u.nm, equivalencies=u.spectral())  

In [ ]:
all_calspec_dict[0]['FLUX'].